In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# ==========================================
# 환경 설정 및 라이브러리 설치
# ==========================================
#!pip uninstall -y torch torchvision torchaudio #import torch안되서 추가한것
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart
!pip install transformers==4.37.2 accelerate pandas tqdm
!pip install transformers torch pandas tqdm
!pip install -q jamo

import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from kobart import get_kobart_tokenizer, get_pytorch_kobart_model
from transformers import BartForConditionalGeneration, AdamW
from tqdm import tqdm
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from google.colab import drive

Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-y6w2ww94/kobart_7bdedaa4ae7d440f9a32b5a5d75746ea
  Running command git clone --filter=blob:none --quiet https://github.com/SKT-AI/KoBART /tmp/pip-install-y6w2ww94/kobart_7bdedaa4ae7d440f9a32b5a5d75746ea
  Resolved https://github.com/SKT-AI/KoBART to commit 79ff1d76ed47550695a05f4cd923642d754b0929
  Preparing metadata (setup.py) ... done


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ 사용 중인 디바이스: {device}")

✅ 사용 중인 디바이스: cuda


### 자모 분리 전처리 함수

In [3]:
import re
import random
from jamo import h2j, j2hcj

# ==========================================
# 3. 자모 분리 함수 정의 (딕셔너리 제거됨)
# ==========================================

def to_jamo_string(s: str) -> str:
    """
    모든 한글 음절을 자모로 분리합니다.
    이전의 딕셔너리 기반 보존 로직을 제거하고, 모든 한글을 분해합니다.
    """
    s = "" if s is None else str(s)
    out = []
    for ch in s:
        # 한글 음절인 경우 무조건 자모로 분해
        if "가" <= ch <= "힣":
            out.append(j2hcj(h2j(ch)))
        # 그 외(영어, 숫자, 특수문자 등)는 그대로 유지
        else:
            out.append(ch)
    return "".join(out)

print("✅ 자모 분리 함수 재정의 완료 (딕셔너리 미사용)")

✅ 자모 분리 함수 재정의 완료 (딕셔너리 미사용)


In [5]:
# ===============================================
# 1. 데이터 불러오기 및 태그 부착 (Data Loading & Tagging)
# ===============================================
path1 = '/content/drive/MyDrive/P01_Nandoc/dataset/일반_10k.csv'
path2 = '/content/drive/MyDrive/P01_Nandoc/dataset/yamin_11k.csv'
path3 = '/content/drive/MyDrive/P01_Nandoc/dataset/new_train_yamin_only.csv'
path4 = '/content/drive/MyDrive/P01_Nandoc/dataset/new_train_normal_only.csv'

print("📂 데이터를 불러오고 태그를 부착합니다...")

if os.path.exists(path1) and os.path.exists(path2):
    # 데이터 로드
    df1 = pd.read_csv(path1)[['input', 'output']]
    df2 = pd.read_csv(path2)[['input', 'output']]
    df3 = pd.read_csv(path3)[['input', 'output']]
    df4 = pd.read_csv(path4)[['input', 'output']]

    # 태그 부착 (Normal/Typo -> <오류>, Yamin/Visual -> <노이즈>)
    # 태그를 한국어로 변경
    df1['input'] = "<오류> " + df1['input'].astype(str)
    df4['input'] = "<오류> " + df4['input'].astype(str)

    df2['input'] = "<노이즈> " + df2['input'].astype(str)
    df3['input'] = "<노이즈> " + df3['input'].astype(str)

    # 데이터 병합 (샘플링 없이 전체 사용)
    df = pd.concat([df1, df2, df3, df4], axis=0).reset_index(drop=True)

    print("✅ 데이터 병합 및 태그 부착 완료")
    print("총 데이터 수:", len(df))
    print("구성:")
    print(f" - <오류> 일반(df1): {len(df1)}")
    print(f" - <오류> 일반(df4): {len(df4)}")
    print(f" - <노이즈> 야민(df2): {len(df2)}")
    print(f" - <노이즈> 야민(df3): {len(df3)}")

    # 예시 출력
    print("\n[데이터 예시]")
    print(df.head())

📂 데이터를 불러오고 태그를 부착합니다...
✅ 데이터 병합 및 태그 부착 완료
총 데이터 수: 32313
구성:
 - <오류> 일반(df1): 11263
 - <오류> 일반(df4): 4906
 - <노이즈> 야민(df2): 11238
 - <노이즈> 야민(df3): 4906

[데이터 예시]
                                               input  \
0  <오류> 뼐 햔 캨뚆 야깝땭. 왜 쌓람뜺러 뼐 1깨를 쯌는찌 껴껴뾴 쌰라므룠셗 먈로...   
1                        <오류> 짬만 짡쿄 칼 때 쭇네요. 쟘옷도 줌 ㅋ   
2                               <오류> 쩔때 꺄면 얀 퇴는 꾯 멗모   
3  <오류> 아... 까껵 초코 쀼됴 뻥 뜛려셔 시원하자만 담배 냄쌢 밁버림. 싸게 하...   
4  <오류> 찌안 츅쳐늚료 따너왔는떼 꺄쎵빠 쬬교 깔끔햐네요. 쩌렴햔뗖 물콰 음료똦 았...   

                                              output  
0  별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자...  
1                             잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ  
2                                    절대 가면 안 되는 곳 메모  
3  아... 가격 좋고 뷰도 뻥 뚫려서 시원하지만 담배 냄새 미쳐버림. 싸게 하루만 묵...  
4  지인 추천으로 다녀왔는데 가성비 좋고 깔끔하네요. 저렴한데 물과 음료도 있고 방도 ...  


### 케이스 B - 자모 분리 생략!
input만 자모로, output은 원문 그대로
“자모 입력을 보고 원문 문장 생성”을 학습

In [6]:
# df["input"] = df["input"].apply(to_jamo_string)
print("✅ 자모 분리 과정을 생략합니다. (원본 텍스트 사용)")
# df["output"]은 그대로 둠

✅ 자모 분리 과정을 생략합니다. (원본 텍스트 사용)


In [7]:
print("입력(Input):", df.loc[0, "input"])
# print("자모:", to_jamo_string(df.loc[0, "input"]))
print("정답(Output):", df.loc[0, "output"])

입력(Input): <오류> 뼐 햔 캨뚆 야깝땭. 왜 쌓람뜺러 뼐 1깨를 쯌는찌 껴껴뾴 쌰라므룠셗 먈로 쎨명햐쨔너 땟글로는 너뮤 껄꾡... 아뮤든 뚜 뼌 따쌌 꺄껄 썷은 꾯. 캠비을 20여 년 따녀뾴 꾯 쭝 쩨일 기쁀 냐뺬뗜 꾯.
정답(Output): 별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자니 댓글로는 너무 길고... 아무튼 두 번 다시 가길 싫은 곳. 캠핑을 20여 년 다녀본 곳 중 제일 기분 나빴던 곳.


In [8]:
# ---------------------------------------------------------
# Train(90%) / Test(10%) 데이터 분리
# ---------------------------------------------------------
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 인덱스 재정렬
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f"✅ 전체 데이터 개수: {len(train_df) + len(test_df)}개")
print(f"   - 학습용 데이터(Train): {len(train_df)}개")
print(f"   - 테스트용 데이터(Test): {len(test_df)}개")

✅ 전체 데이터 개수: 32313개
   - 학습용 데이터(Train): 25850개
   - 테스트용 데이터(Test): 6463개


In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from kobart import get_kobart_tokenizer, get_pytorch_kobart_model
from transformers import BartForConditionalGeneration

# ==========================================
# 2. 데이터셋 클래스 (성능 최적화 적용됨)
# ==========================================
class KoBARTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=256):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        # 패딩 토큰 ID 미리 저장 (KoBART는 보통 3)
        self.pad_token_id = tokenizer.pad_token_id

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        input_text = str(row['input'])
        target_text = str(row['output'])

        inputs = self.tokenizer(
            input_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        targets = self.tokenizer(
            target_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        labels = targets['input_ids'].squeeze()

        # 🔥 [필수 추가] 패딩 토큰은 채점하지 마! (-100 처리)
        # 이걸 넣어야 모델이 정답 문장 생성에만 집중합니다.
        labels[labels == self.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'raw_input': input_text,
            'raw_output': target_text
        }

print("🔄 토크나이저 및 모델 로딩 중...")
tokenizer = get_kobart_tokenizer()
model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model())

# ---------------------------------------------------------
# 🔥 [설정] 스페셜 토큰 추가 (<오류>, <노이즈>)
# ---------------------------------------------------------
# 전처리 단계에서 사용한 태그와 100% 일치해야 합니다.
special_tokens = {'additional_special_tokens': ['<오류>', '<노이즈>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens)

if num_added_toks > 0:
    print(f"✅ 스페셜 토큰 추가됨: {special_tokens}")
    model.resize_token_embeddings(len(tokenizer))
    print(f"✅ 모델 임베딩 사이즈 조정 완료: {len(tokenizer)}")

    # ---------------------------------------------------------
    # 🚀 [부스터] 스페셜 토큰 임베딩 똑똑하게 초기화하기 (수정됨)
    # ---------------------------------------------------------
    print("🔄 태그 임베딩 초기화(Smart Init) 수행 중...")
    with torch.no_grad():
        # 1. <오류>는 '수정'이라는 단어의 뜻을 카피
        if '<오류>' in special_tokens['additional_special_tokens']:
            err_idx = tokenizer.convert_tokens_to_ids('<오류>')

            # 🔥 [수정 포인트] add_special_tokens=False로 순수 단어 토큰만 가져옴
            ref_ids = tokenizer.encode('수정', add_special_tokens=False)
            if len(ref_ids) > 0:
                ref_idx = ref_ids[0] # 첫 번째 토큰 사용
                model.model.shared.weight[err_idx] = model.model.shared.weight[ref_idx]
            else:
                print("⚠️ 경고: '수정'이라는 단어를 인코딩하지 못했습니다. 초기화 건너뜀.")

        # 2. <노이즈>는 '해석'이라는 단어의 뜻을 카피
        if '<노이즈>' in special_tokens['additional_special_tokens']:
            noise_idx = tokenizer.convert_tokens_to_ids('<노이즈>')

            # 🔥 [수정 포인트] 여기도 동일하게 적용
            ref_ids = tokenizer.encode('해석', add_special_tokens=False)
            if len(ref_ids) > 0:
                ref_idx = ref_ids[0]
                model.model.shared.weight[noise_idx] = model.model.shared.weight[ref_idx]
            else:
                print("⚠️ 경고: '해석'이라는 단어를 인코딩하지 못했습니다. 초기화 건너뜀.")

    print("✅ 임베딩 초기화 완료! 학습 준비 끝.")


model.to(device)

# 학습용 데이터셋 생성
train_dataset = KoBARTDataset(train_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 테스트용 데이터셋 생성
test_dataset = KoBARTDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


🔄 토크나이저 및 모델 로딩 중...
downloading model from https://huggingface.co/skt/kobart-base-v1/resolve/main/legacy/kobart_base_tokenizer_cased_cf74400bce.zip...
downloading model from https://huggingface.co/skt/kobart-base-v1/resolve/main/legacy/kobart_base_cased_ff4bda5738.zip...


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


✅ 스페셜 토큰 추가됨: {'additional_special_tokens': ['<오류>', '<노이즈>']}
✅ 모델 임베딩 사이즈 조정 완료: 30002
🔄 태그 임베딩 초기화(Smart Init) 수행 중...
✅ 임베딩 초기화 완료! 학습 준비 끝.


In [10]:
# ==========================================
# 3. 학습 (Train Loop)
# ==========================================
optimizer = AdamW(model.parameters(), lr=3e-5)
epochs = 15

print(f"🚀 학습 시작 (총 {epochs} Epochs)...")
model.train()
for epoch in range(epochs):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    total_loss = 0
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        raw_texts = list(batch["raw_input"]) # 배치의 원문 리스트
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} 평균 Loss: {total_loss / len(train_loader):.4f}")

🚀 학습 시작 (총 15 Epochs)...


Epoch 1/15: 100%|██████████| 1616/1616 [06:25<00:00,  4.19it/s, loss=0.808]


Epoch 1 평균 Loss: 1.3657


Epoch 2/15: 100%|██████████| 1616/1616 [06:25<00:00,  4.20it/s, loss=0.7]


Epoch 2 평균 Loss: 0.5854


...

Epoch 13 평균 Loss: 0.0390


Epoch 14/15: 100%|██████████| 1616/1616 [06:25<00:00,  4.19it/s, loss=0.0121]


Epoch 14 평균 Loss: 0.0318


Epoch 15/15: 100%|██████████| 1616/1616 [06:25<00:00,  4.19it/s, loss=0.00227]

Epoch 15 평균 Loss: 0.0269


# 뇌절방지 후처리 인퍼런스 코드

In [32]:
import re
import os
import torch
import pandas as pd
from jiwer import cer
from bert_score import score
from tqdm.auto import tqdm

# =========================================================
# 0. 💾 저장 경로 설정
# =========================================================
model_save_path = '/content/drive/MyDrive/P01_Nandoc/FINALV0/'
# 폴더가 혹시 없으면 생성
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

print(f"📂 저장 경로 확인: {model_save_path}")

# =========================================================
# 1.  이중 잠금 컷 (Double Lock Strategy)
# =========================================================
def double_lock_cut(input_text, pred_text):
    # [Lock 1] 문장 개수 동기화
    # 입력에 있는 문장 부호(. ! ?) 개수만큼만 출력에서도 살리기
    input_punctuations = len(re.findall(r'[.!?]', input_text))
    target_count = max(input_punctuations, 1)

    matches = [m.start() for m in re.finditer(r'[.!?]', pred_text)]

    if len(matches) >= target_count:
        cut_idx = matches[target_count - 1]
        candidate = pred_text[:cut_idx+1]
    else:
        candidate = pred_text

    # [Lock 2] 길이 제한 (입력의 1.3배 + 10자)
    # 문장 개수를 맞췄더라도, 내용이 너무 길어지면 강제로 자르기
    limit_len = int(len(input_text) * 1.3) + 10

    if len(candidate) > limit_len:
        truncated = candidate[:limit_len]
        last_punc = max(truncated.rfind('.'), truncated.rfind('!'), truncated.rfind('?'))
        if last_punc != -1:
            return truncated[:last_punc+1]
        last_space = truncated.rfind(' ')
        if last_space != -1:
            return truncated[:last_space]
        return truncated

    return candidate

# =========================================================
# 2. 최종 정제 (Final Polish)
# =========================================================
def final_polish(text):
    # 입력에 있는 문장 부호(. ! ?) 개수만큼만 출력에서도 살리기
    clean_pattern = r'[^가-힣a-zA-Z0-9\s.,?!\'"%~]'
    text = re.sub(clean_pattern, '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 다중 공백 및 중복 문장 제거
    sentences = re.split(r'(?<=[.?!])\s+', text)
    unique_sentences = []
    for s in sentences:
        s = s.strip()
        if not s: continue
        if unique_sentences and s == unique_sentences[-1]:
            continue
        unique_sentences.append(s)
    return ' '.join(unique_sentences)

# =========================================================
# 3. 🚀 전체 데이터 추론 (Full Inference)
# =========================================================
print(f"🚀 전체 데이터({len(test_loader.dataset)}개) 추론 및 평가 시작...")

# 파라미터 확정
model.config.update({
    "num_beams": 5,
    "no_repeat_ngram_size": 3,
    "repetition_penalty": 1.5,
    "early_stopping": True,
    "eos_token_id": tokenizer.eos_token_id,
    "max_length": 256
})
model.eval()

result_data = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Processing"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        raw_inputs = batch['raw_input']
        raw_outputs = batch['raw_output']

        # 모델 생성
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        # 후처리
        for i, text in enumerate(generated_texts):
            cut_text = double_lock_cut(raw_inputs[i], text)
            final_text = final_polish(cut_text)

            result_data.append({
                "input": raw_inputs[i],
                "ground_truth": raw_outputs[i],
                "prediction": final_text
            })


📂 저장 경로 확인: /content/drive/MyDrive/P01_Nandoc/FINALV0/
🚀 전체 데이터(6463개) 추론 및 평가 시작...


Processing:   0%|          | 0/202 [00:00<?, ?it/s]

In [35]:
# ==========================================
# 💾 학습된 모델 및 토크나이저 저장
# ==========================================
import os

# 저장 경로 재확인
model_save_path = '/content/drive/MyDrive/P01_Nandoc/FINALV0/'

if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

print(f"💾 모델과 토크나이저를 저장합니다...\n경로: {model_save_path}")

# 모델 저장
model.save_pretrained(model_save_path)
# 토크나이저 저장
tokenizer.save_pretrained(model_save_path)

print("✅ 저장 완료! 이제 불러오기 코드가 정상 작동할 것입니다.")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'min_length': 10, 'early_stopping': True, 'num_beams': 5, 'repetition_penalty': 1.5, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 1}


💾 모델과 토크나이저를 저장합니다...
경로: /content/drive/MyDrive/P01_Nandoc/FINALV0/
✅ 저장 완료! 이제 불러오기 코드가 정상 작동할 것입니다.


In [33]:

# =========================================================
# 4. 📊 최종 평가 (CER + BERTScore)
# =========================================================
df_result = pd.DataFrame(result_data)

# 저장 경로
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)
output_path = f"{model_save_path}/output_test_results.csv"

# CSV 저장
df_result.to_csv(output_path, index=False, encoding="utf-8-sig")

print("\n" + "="*60)
print("🏆 최종 모델 성능 리포트")
print("="*60)

# 결측치 제거
df_clean = df_result.dropna(subset=['ground_truth', 'prediction'])
preds = df_clean['prediction'].astype(str).tolist()
refs = df_clean['ground_truth'].astype(str).tolist()

# 1. CER 계산
try:
    final_cer = cer(refs, preds)
    print(f"📊 CER (낮을수록 좋음): {final_cer:.4f}")
except Exception as e:
    print(f"⚠️ CER 계산 실패: {e}")

# 2. BERTScore 계산
print("\n⏳ BERTScore 계산 중... (잠시만 기다려주세요)")
try:
    # lang="ko" 설정으로 한국어 모델 자동 로드
    P, R, F1 = score(preds, refs, lang="ko", verbose=True)

    print(f"📊 BERTScore - Precision: {P.mean().item():.4f}")
    print(f"📊 BERTScore - Recall:    {R.mean().item():.4f}")
    print(f"📊 BERTScore - F1:        {F1.mean().item():.4f}")
    print("   (F1 점수가 1.0에 가까울수록 문맥이 완벽함)")
except Exception as e:
    print(f"⚠️ BERTScore 계산 실패: {e}")

print("="*60)
print(f"💾 최종 결과 파일: {output_path}")


🏆 최종 모델 성능 리포트
📊 CER (낮을수록 좋음): 0.1569

⏳ BERTScore 계산 중... (잠시만 기다려주세요)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


calculating scores...
computing bert embedding.


  0%|          | 0/162 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/101 [00:00<?, ?it/s]

done in 24.57 seconds, 263.01 sentences/sec
📊 BERTScore - Precision: 0.9586
📊 BERTScore - Recall:    0.9507
📊 BERTScore - F1:        0.9543
   (F1 점수가 1.0에 가까울수록 문맥이 완벽함)
💾 최종 결과 파일: /content/drive/MyDrive/P01_Nandoc/FINALV0//output_test_results.csv


# 아래는 실험, 전략 반영후 테스트.

[데이터 처리 파이프라인]

Input: 난독화된 텍스트 입력

KoBART: 문맥을 파악하여 1차 복원 (여기까진 뇌절 포함)

Lock 1 (Count Sync): 입력의 마침표 개수만큼만 딱 끊어냄 (대부분의 뇌절 제거)

Lock 2 (Length Limit): 그래도 남은 긴 꼬리를 1.3배 길이에서 물리적으로 차단

Polish: 중복 문장 및 특수문자 찌꺼기 제거

In [28]:
import re
import torch
import pandas as pd
from jiwer import cer
from tqdm.auto import tqdm

# =========================================================
# 1. ✂️ 이중 잠금 컷 함수 (Double Lock: Count + Length)
# =========================================================
def double_lock_cut(input_text, pred_text):
    # --- [1차 잠금] 문장 개수 동기화 ---
    # 입력의 문장 부호(. ! ?) 개수 확인
    input_punctuations = len(re.findall(r'[.!?]', input_text))
    target_count = max(input_punctuations, 1) # 최소 1문장은 있다고 가정

    # 예측의 문장 부호 위치 확인
    matches = [m.start() for m in re.finditer(r'[.!?]', pred_text)]

    # 문장 부호 개수가 입력보다 많거나 같으면, 해당 위치에서 컷
    if len(matches) >= target_count:
        cut_idx = matches[target_count - 1]
        candidate = pred_text[:cut_idx+1]
    else:
        # 부호가 모자르면 일단 원본 유지 (2차 잠금에서 처리)
        candidate = pred_text

    # --- [2차 잠금] 강력한 길이 제한 (1.3배) ---
    # 문장 개수를 맞췄더라도, 혹은 못 맞췄더라도 절대적 길이 제한 적용
    limit_len = int(len(input_text) * 1.3) + 10

    if len(candidate) > limit_len:
        # 길이 초과 시, limit_len 안쪽에서 가장 마지막 공백/부호에서 자름
        truncated = candidate[:limit_len]

        # 1순위: 문장 부호(.!?)
        last_punc = max(truncated.rfind('.'), truncated.rfind('!'), truncated.rfind('?'))
        if last_punc != -1:
            return truncated[:last_punc+1]

        # 2순위: 공백
        last_space = truncated.rfind(' ')
        if last_space != -1:
            return truncated[:last_space]

        return truncated

    return candidate

# =========================================================
# 2. ✨ 최종 정제 함수 (Cleaning)
# =========================================================
def final_polish(text):
    # 1. 특수문자 제거 (한글, 영어, 숫자, 기본 부호만 남김)
    clean_pattern = r'[^가-힣a-zA-Z0-9\s.,?!\'"%~]'
    text = re.sub(clean_pattern, '', text)

    # 2. 다중 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()

    # 3. 문장 중복 제거 (Dedup)
    sentences = re.split(r'(?<=[.?!])\s+', text)
    unique_sentences = []
    for s in sentences:
        s = s.strip()
        if not s: continue
        if unique_sentences and s == unique_sentences[-1]:
            continue
        unique_sentences.append(s)

    return ' '.join(unique_sentences)

# =========================================================
# 3. 🧪 샘플 10개 테스트 (Double Lock + Polish)
# =========================================================
print("🧪 [Grand Final] 샘플 10개 테스트 시작 (Double Lock)...")

# 파라미터 재설정 (확실하게)
model.config.update({
    "num_beams": 5,
    "no_repeat_ngram_size": 3,
    "repetition_penalty": 1.5,
    "early_stopping": True,
    "eos_token_id": tokenizer.eos_token_id,
    "max_length": 256
})
model.eval()

result_data = []
target_count = 10
current_count = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        raw_inputs = batch['raw_input']
        raw_outputs = batch['raw_output']

        # 생성
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        for i, text in enumerate(generated_texts):
            # 1. 이중 잠금 컷
            cut_text = double_lock_cut(raw_inputs[i], text)

            # 2. 최종 정제
            final_text = final_polish(cut_text)

            result_data.append({
                "input": raw_inputs[i],
                "ground_truth": raw_outputs[i],
                "prediction": final_text
            })

            current_count += 1
            if current_count >= target_count:
                break

        if current_count >= target_count:
            break

# =========================================================
# 4. 결과 확인
# =========================================================
df_sample = pd.DataFrame(result_data)
preds = df_sample['prediction'].astype(str).tolist()
refs = df_sample['ground_truth'].astype(str).tolist()

print("\n" + "="*60)
print(f"📊 [Double Lock Sample Result]")
print(f"📊 CER: {cer(refs, preds):.4f}")
print("="*60 + "\n")

for i in range(len(preds)):
    print(f"입력: {df_sample.iloc[i]['input']}")
    print(f"예측: {preds[i]}")
    print(f"정답: {refs[i]}")
    print("-" * 50)

🧪 [Grand Final] 샘플 10개 테스트 시작 (Double Lock)...

📊 [Double Lock Sample Result]
📊 CER: 0.2563

입력: <노이즈> ㅋr운터O|| 4장님0l 너ㅁ 불쾌하고 반말ㅎr셨어dD. 헤어 드ㄹre717r 없어요.
예측: 카운터에 사장님이 너무 불쾌하고 반말하셨어요. 헤어 드라이기가 없어요.
정답: 카운터에 사장님이 너무 불쾌하고 반말하셨어요. 헤어 드라이기가 없어요.
--------------------------------------------------
입력: <오류> 뱌땼갸 뽀어는 뷰꺄 몃잡넏탸. 빠있 깨끗함닎탘. 바다를 뽀며 쩐끼 크릴에 고키를 꾸유니 좋씀닗다. 냉난바어 쨜뙤는 것 같습니따. 뺘타 쬭의 정원이 깨끗하꼬 예쁘네오. 주편 몇 깨의 펜셔니 까터 싸용햐는 뜻하네요. 여기만 산책해도 좋습니탸.
예측: 바다가 보이는 뷰가 멋집니다. 방이 깨끗합니다. 바다를 보며 전기 그릴에 고기를 구우니 좋습니다. 냉난방이 잘되는 것 같습니다. 바다 쪽의 정원이 깨끗하고 예쁘네요. 주변 몇 개의 펜션이 같이 사용하는 듯하네요. 여기만 산책해도 좋습니다.
정답: 바다가 보이는 뷰가 멋집니다. 방이 깨끗합니다. 바다를 보며 전기 그릴에 고기를 구우니 좋습니다. 냉난방이 잘되는 것 같습니다. 바다 쪽의 정원이 깨끗하고 예쁘네요. 주변 몇 개의 펜션이 같이 사용하는 듯하네요. 여기만 산책해도 좋습니다.
--------------------------------------------------
입력: <오류> 이번에 친구랑 같이 부산 해운대에 다녀왔는데, 숙소가 기대 아상어었어오. 방은 넓코 아담해서 4명이 함께 이써도 불편함이 없었고, 창밖으로 뽀이는 바다 쩐망이 알퓸이었어요. 바딱은 스파어쓰 바댜기라 걸을 때맑다 부드럽게 감싸주는 느낌어었어요. 방에 삐치된 커피 머선은 스타벅스 캍은 고급스러웠코, 샴푸·컨디셔너는 찬환경 프랜드라서 피부에 좋았여요. 공용 랊운치에는 작은 캬페도 있어서 아치메 커퍼 한 잔 마시며 

In [30]:
import re
import torch
import pandas as pd
from jiwer import cer
from tqdm.auto import tqdm

# =========================================================
# 1. ✂️ 기존 안전장치 (각 조각이 뇌절 못하게 막음)
# =========================================================
def double_lock_cut(input_text, pred_text):
    # [Lock 1] 문장 부호 개수 동기화
    input_punctuations = len(re.findall(r'[.!?]', input_text))
    target_count = max(input_punctuations, 1)
    matches = [m.start() for m in re.finditer(r'[.!?]', pred_text)]

    if len(matches) >= target_count:
        cut_idx = matches[target_count - 1]
        candidate = pred_text[:cut_idx+1]
    else:
        candidate = pred_text

    # [Lock 2] 길이 제한 (1.3배)
    limit_len = int(len(input_text) * 1.3) + 10
    if len(candidate) > limit_len:
        truncated = candidate[:limit_len]
        last_punc = max(truncated.rfind('.'), truncated.rfind('!'), truncated.rfind('?'))
        if last_punc != -1: return truncated[:last_punc+1]
        last_space = truncated.rfind(' ')
        if last_space != -1: return truncated[:last_space]
        return truncated
    return candidate

def final_polish(text):
    clean_pattern = r'[^가-힣a-zA-Z0-9\s.,?!\'"%~]'
    text = re.sub(clean_pattern, '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# =========================================================
# 2. 🧠 [핵심] 스마트 반갈죽 로직 (Half-Split Strategy)
# =========================================================
def smart_split_inference(text, model, tokenizer, device):
    # 1. 짧은 문장(100자 미만)은 그냥 한 번에 처리 (자르는 게 오히려 손해)
    if len(text) < 100:
        return run_single_inference(text, model, tokenizer, device)

    # 2. 긴 문장은 "공백"을 기준으로 반으로 가름
    else:
        mid_idx = len(text) // 2

        # 중간 지점에서 가장 가까운 공백 찾기 (단어 중간 절단 방지)
        # 왼쪽으로 탐색
        left_space = text.rfind(' ', 0, mid_idx)
        # 오른쪽으로 탐색
        right_space = text.find(' ', mid_idx)

        # 더 가까운 쪽 선택
        split_idx = mid_idx
        if left_space == -1 and right_space == -1:
            split_idx = mid_idx # 공백이 없으면 강제 절단
        elif left_space == -1:
            split_idx = right_space
        elif right_space == -1:
            split_idx = left_space
        else:
            # mid와의 거리가 더 가까운 쪽 선택
            if (mid_idx - left_space) < (right_space - mid_idx):
                split_idx = left_space
            else:
                split_idx = right_space

        # ✂️ 분리!
        part1 = text[:split_idx].strip()
        part2 = text[split_idx:].strip()

        # 🏃‍♂️ 각각 모델 태우기 (서로 다른 문장인 척)
        res1 = run_single_inference(part1, model, tokenizer, device)
        res2 = run_single_inference(part2, model, tokenizer, device)

        # 🔗 합체
        return (res1 + " " + res2).strip()

def run_single_inference(text, model, tokenizer, device):
    """작은 조각 하나를 추론하는 함수"""
    if not text: return ""

    inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True).to(device)

    with torch.no_grad():
        gen_ids = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            num_beams=5,
            no_repeat_ngram_size=3,
            repetition_penalty=1.5,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id,
            max_length=128 # 조각이 작으니까 max_length도 작게 잡음
        )
    pred = tokenizer.decode(gen_ids[0], skip_special_tokens=True)

    # 각 조각에도 안전장치 적용
    cut_pred = double_lock_cut(text, pred)
    return final_polish(cut_pred)

# =========================================================
# 3. 🧪 샘플 10개 테스트 (반갈죽 적용)
# =========================================================
print("🧪 [Smart Half-Split] 샘플 10개 테스트 시작...")
model.eval()

result_data = []
target_count = 10
current_count = 0

with torch.no_grad():
    for batch in test_loader:
        raw_inputs = batch['raw_input']
        raw_outputs = batch['raw_output']

        for i, input_text in enumerate(raw_inputs):
            # 🔥 시훈님의 아이디어 적용!
            final_pred = smart_split_inference(input_text, model, tokenizer, device)

            result_data.append({
                "input": input_text,
                "ground_truth": raw_outputs[i],
                "prediction": final_pred
            })

            current_count += 1
            if current_count >= target_count: break
        if current_count >= target_count: break

# 결과 확인
df_sample = pd.DataFrame(result_data)
preds = df_sample['prediction'].astype(str).tolist()
refs = df_sample['ground_truth'].astype(str).tolist()

print("\n" + "="*60)
print(f"📊 [Smart Half-Split Result]")
print(f"📊 CER: {cer(refs, preds):.4f}")
print("="*60 + "\n")

for i in range(len(preds)):
    print(f"입력: {df_sample.iloc[i]['input']}")
    print(f"예측: {preds[i]}")
    print(f"정답: {refs[i]}")
    print("-" * 50)

🧪 [Smart Half-Split] 샘플 10개 테스트 시작...

📊 [Smart Half-Split Result]
📊 CER: 0.3291

입력: <노이즈> ㅋr운터O|| 4장님0l 너ㅁ 불쾌하고 반말ㅎr셨어dD. 헤어 드ㄹre717r 없어요.
예측: 카운터에 사장님이 너무 불쾌하고 반말하셨어요. 헤어 드라이기가 없어요.
정답: 카운터에 사장님이 너무 불쾌하고 반말하셨어요. 헤어 드라이기가 없어요.
--------------------------------------------------
입력: <오류> 뱌땼갸 뽀어는 뷰꺄 몃잡넏탸. 빠있 깨끗함닎탘. 바다를 뽀며 쩐끼 크릴에 고키를 꾸유니 좋씀닗다. 냉난바어 쨜뙤는 것 같습니따. 뺘타 쬭의 정원이 깨끗하꼬 예쁘네오. 주편 몇 깨의 펜셔니 까터 싸용햐는 뜻하네요. 여기만 산책해도 좋습니탸.
예측: 바다가 보이는 뷰가 멋집니다. 방이 깨끗합니다. 바다를 보며 전기 그릴에 고기를 구우니 좋습니다. 바다 쪽 정원이 깨끗하고 예쁘네요. 주변 몇 개의 펜션이 같이 사용하는 듯하네요. 여기만 산책해도 좋습니다. 좋습니다.
정답: 바다가 보이는 뷰가 멋집니다. 방이 깨끗합니다. 바다를 보며 전기 그릴에 고기를 구우니 좋습니다. 냉난방이 잘되는 것 같습니다. 바다 쪽의 정원이 깨끗하고 예쁘네요. 주변 몇 개의 펜션이 같이 사용하는 듯하네요. 여기만 산책해도 좋습니다.
--------------------------------------------------
입력: <오류> 이번에 친구랑 같이 부산 해운대에 다녀왔는데, 숙소가 기대 아상어었어오. 방은 넓코 아담해서 4명이 함께 이써도 불편함이 없었고, 창밖으로 뽀이는 바다 쩐망이 알퓸이었어요. 바딱은 스파어쓰 바댜기라 걸을 때맑다 부드럽게 감싸주는 느낌어었어요. 방에 삐치된 커피 머선은 스타벅스 캍은 고급스러웠코, 샴푸·컨디셔너는 찬환경 프랜드라서 피부에 좋았여요. 공용 랊운치에는 작은 캬페도 있어서 아치메 커퍼 한 잔 마시며 일상을 계획할 수 있었꼬, 쥬방 시설도 완

# 손으로 실험해보자

In [36]:
import os
import torch
import re
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# ==========================================
# 1.  방어 로직 정의 (이중 잠금 + 정제)
# ==========================================
def double_lock_cut(input_text, pred_text):
    """
    1차: 문장 부호 개수 맞추기
    2차: 입력 길이의 1.3배 넘으면 자르기
    """
    # [Lock 1] 문장 개수 동기화
    input_punctuations = len(re.findall(r'[.!?]', input_text))
    target_count = max(input_punctuations, 1)

    matches = [m.start() for m in re.finditer(r'[.!?]', pred_text)]
    if len(matches) >= target_count:
        cut_idx = matches[target_count - 1]
        candidate = pred_text[:cut_idx+1]
    else:
        candidate = pred_text

    # [Lock 2] 길이 제한 (1.3배)
    limit_len = int(len(input_text) * 1.3) + 10
    if len(candidate) > limit_len:
        truncated = candidate[:limit_len]
        last_punc = max(truncated.rfind('.'), truncated.rfind('!'), truncated.rfind('?'))
        if last_punc != -1: return truncated[:last_punc+1]
        last_space = truncated.rfind(' ')
        if last_space != -1: return truncated[:last_space]
        return truncated
    return candidate

def final_polish(text):
    """특수문자 찌꺼기 및 중복 문장 제거"""
    clean_pattern = r'[^가-힣a-zA-Z0-9\s.,?!\'"%~]'
    text = re.sub(clean_pattern, '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    sentences = re.split(r'(?<=[.?!])\s+', text)
    unique_sentences = []
    for s in sentences:
        s = s.strip()
        if not s: continue
        if unique_sentences and s == unique_sentences[-1]:
            continue
        unique_sentences.append(s)
    return ' '.join(unique_sentences)

# ==========================================
# 2. 모델 로딩
# ==========================================
# 저장된 모델 경로 (최종 성능 좋았던 경로 사용)
model_path = '/content/drive/MyDrive/P01_Nandoc/FINALV0/'

print(f"📂 저장된 모델을 불러오는 중입니다... ({model_path})")

if os.path.exists(model_path):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
    model = BartForConditionalGeneration.from_pretrained(model_path)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print("✅ 모델 로딩 완료! (Double Lock 파이프라인 장착됨)")
else:
    print(f"❌ 모델 파일이 없습니다: {model_path}")
    print("   경로를 확인하거나, 학습 코드를 먼저 실행해서 모델을 저장해주세요.")
    exit()

# ==========================================
# 3. 사용자 입력 및 복원 실행
# ==========================================
def restore_text(text):
    # 입력 텍스트 토큰화
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # 모델 생성 (파라미터도 최적값으로 수정함)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=256,
            num_beams=5,
            no_repeat_ngram_size=3,
            repetition_penalty=1.5,  # 뇌절 방지 핵심 파라미터
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id
        )

    # 1. Raw Output 디코딩
    raw_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # 2. 파이프라인 적용 (이게 없으면 테스트 때도 뇌절함)
    cut_text = double_lock_cut(text, raw_output)
    final_text = final_polish(cut_text)

    return final_text

# ==========================================
# 4. 실행 루프
# ==========================================
print("\n" + "="*50)
print("💡 [야민정음/오타 복원기] 시작 (종료: 'q' 입력)")
print("   * 이중 잠금 & 정제 로직이 적용되어 있습니다.")
print("="*50)

while True:
    input_text = input("\n📝 문장을 입력하세요: ")

    if input_text.lower() in ['q', 'quit', 'exit']:
        print("👋 프로그램을 종료합니다.")
        break

    if not input_text.strip():
        continue

    # 복원 수행
    restored = restore_text(input_text)

    print("-" * 50)
    print(f"▶️ 원본: {input_text}")
    print(f"✨ 복원: {restored}")
    print("-" * 50)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


📂 저장된 모델을 불러오는 중입니다... (/content/drive/MyDrive/P01_Nandoc/FINALV0/)
✅ 모델 로딩 완료! (Double Lock 파이프라인 장착됨)

💡 [야민정음/오타 복원기] 시작 (종료: 'q' 입력)
   * 이중 잠금 & 정제 로직이 적용되어 있습니다.

📝 문장을 입력하세요: 거난 즉말O|| 강릉O|| 있= ‘ㅂr㉰ 파당’ gays트Ha윽sO|| 2박했=데, bㅁ- 전망은 맘O|| dl었거만 직원ㅇl 안경 끼그 ‘겨71yO’ㄹr끄만 인saㅎri서 외국인 친7㉠r ‘어OI7, 서네s㉠r 어딨어?’ ㅎr끄 말했어. 조싀은 새윽0a 솓불끄712r서 먹ヱ 싶었는데 ㅅㅑ워71 물ㅇl 끊김ㅇI ㅈr꾸 일어Lㅏ그 욕실 ㅌr일O|| 물때7r 남Oㅏ 있어서 조금 불편했어. ㉠r격은 1박 12,000원 경든ro ㅁ-른 곳보ㅁ- 13싸저 않긴 한데, 청Show㉠r 그ㅁ-겨 깨끗ㅎr저 않Oㅏ Hbang문은 없겠저. 그2H든 hozl ㅁ렵 ㅂr㉰를 보띠 맥즉 한 잔은 좋은 추억01었어.
--------------------------------------------------
▶️ 원본: 거난 즉말O|| 강릉O|| 있= ‘ㅂr㉰ 파당’ gays트Ha윽sO|| 2박했=데, bㅁ- 전망은 맘O|| dl었거만 직원ㅇl 안경 끼그 ‘겨71yO’ㄹr끄만 인saㅎri서 외국인 친7㉠r ‘어OI7, 서네s㉠r 어딨어?’ ㅎr끄 말했어. 조싀은 새윽0a 솓불끄712r서 먹ヱ 싶었는데 ㅅㅑ워71 물ㅇl 끊김ㅇI ㅈr꾸 일어Lㅏ그 욕실 ㅌr일O|| 물때7r 남Oㅏ 있어서 조금 불편했어. ㉠r격은 1박 12,000원 경든ro ㅁ-른 곳보ㅁ- 13싸저 않긴 한데, 청Show㉠r 그ㅁ-겨 깨끗ㅎr저 않Oㅏ Hbang문은 없겠저. 그2H든 hozl ㅁ렵 ㅂr㉰를 보띠 맥즉 한 잔은 좋은 추억01었어.
✨ 복원: 지난 주말에 강릉에 있는 바다 마당 게스트하우스에 2박했는데, 바다 전망은 맘에 들었지만 직원이 안경 끼고 저기요라고만 인사해서 외국인 친구가 어이구,

KeyboardInterrupt: Interrupted by user